**In case** there is issue with reloading, run the following to delete compiled Python cache.

This is **typically** not needed.
Often, restarting the notebook works better.

In [ ]:
!cd ..; cd arguments; rm -rf __pycache__
!cd ..; cd constants; rm -rf __pycache__
!cd ..; cd data; rm -rf __pycache__
!cd ..; cd logger; rm -rf __pycache__
!cd ..; cd models; rm -rf __pycache__
!cd ..; cd utils; rm -rf __pycache__

In [ ]:
!hostname

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Install gcloud if not already there
!pip install lightgbm --user

In [ ]:
import os

root = os.environ['HOME'] + '/temp_store/candle_60_data/AUDUSD'
days = sorted(os.listdir(root))
for day in days:
    print(day)
    lps = sorted(os.listdir(root + '/' + day))
    if len(lps) != 5:
        print('  MISSING LP!')
    for f in sorted(os.listdir(root + '/' + day)):
        print(f'\t{f}')

# Entrance Notebook for Forex Trading

This is for testing only. For batched runs a main file is still required.

Keep autoreload for faster Python module reloading.

## Imports

Import Python default packges first, then those requiring `pip`, finally modules in the code structure. Keep alphabetical order.

In [ ]:
import sys
sys.path.append('..')

import os

import pandas as pd
import tqdm
from sklearn.metrics import make_scorer

from arguments import *
from constants import *
from data import *
import models as supported_models
from training_tools import *
from utils import *

## Preparations

* Setup interface with Google Cloud bucket.
* Pretend to have command line argument.
* Prepare data
* Define models

In [ ]:
def gether_data(loader):
    all_x, all_gt = [], []
    total_len = len(loader.dataset)
    for i in range(total_len):
        x, gt = loader.dataset[i]
        if i % 500 == 0:
            logger.log_stdout(f'# [{i + 1}]/{total_len}')

        x = x.flatten()
        # gt = gt

        all_x.append(x)
        all_gt.append(gt)
    
    return all_x, all_gt

In [ ]:
storage = GCStorage.get_CloudFS(project_name=PROJECT_NAME,
                                bucket_name=GC_BUCKET,
                                credential_path=CREDENTIAL_PATH)

In [ ]:
all_args = parse_from_string('--exp_name=ml --device=cpu '
                             '--currency_pair=EURUSD '
                             '--num_candles=4 --candle_interval=300 '
                             '--num_workers=1 '
                             '--log_level=4 '
                             '--exp_setting=simple_regression')

logger = all_args.misc_args.logger
train_loader, valid_loader = get_dataloaders(all_args)
loaders = {'train': train_loader, 'valid': valid_loader}

In [ ]:
train_x, train_gt = gether_data(train_loader)
valid_x, valid_gt = gether_data(valid_loader)

## Train (Machine Learning) (Tweak One Model with GridSearchCV)

Here we only use the tools to gather data :P

In [ ]:
import pprint as pp
from sklearn.model_selection import GridSearchCV

In [ ]:
if ALL_SETTINGS[all_args.misc_args.exp_setting]['type'] == 'regression':
    regressors_list = supported_models.regressors
    metric_list = supported_models.regression_metrics
    all_models = {model_name: regressors_list[model_name] \
                                                for model_name in regressors_list}
elif ALL_SETTINGS[all_args.misc_args.exp_setting]['type'] == 'binary_classification':
    classifier_list = supported_models.classifiers
    metric_list = supported_models.binary_classification_metrics
    all_models = {model_name: classifier_list[model_name] \
                                                for model_name in classifier_list}

In [ ]:
model_name = 'kernel_ridge'
model = all_models[model_name]

'''
param_dict = {'min_child_samples': [0, 2],
               'class_weight':['balanced'],
               'max_depth': [4, 6, 8],
               'num_leaves': [31, 63, 127],
               'min_split_gain': [0],
               'reg_alpha': [0.01, 0.05],
               'reg_lambda': [0.6, 0.7,],
               'n_estimators': [64, 128]}
'''
param_dict = {'alpha': [0.01, 0.1, 1], 'kernel': ['linear']}
# param_dict = {}

scoring_fn = make_scorer(
            metric_list['kendaltau'][0],
            greater_is_better=metric_list['kendaltau'][1])

model = GridSearchCV(model, param_dict, scoring=scoring_fn, cv=5, refit=True, n_jobs=28, verbose=10)
print(model)

In [ ]:
model.fit(train_x, train_gt)

In [ ]:
fit_df = pd.DataFrame(model.cv_results_)
fit_df[[c for c in fit_df.columns if 'param' in c or 'mean_test_score' in c]].sort_values(['mean_test_score'], ascending=False)

In [ ]:
def generate_confidence_interval(ground_truth, predictions,
                                 score_func,
                                 num_trials=500,
                                 confidence_level=0.95):
    '''Generate confidence interval given ground truth and prediction.
    
    Bootstrapping to get confidence interval for prediction.

    Argument:
        data (dict): ground truth and predictions
        num_trials (int): number of trials for bootstrapping
        confidence_level (float): confidence level
    
    Return:
        dict({lb: float, mean: float, ub: float}): confidence interval
    '''
    
    
    ground_truth = np.array(ground_truth)
    predictions = np.array(predictions)

    scores = []
    num_successes = 0
    num_tries = 0
    indices = list(range(len(ground_truth)))

    while (num_successes < num_trials):
        # Limit the number of tries.
        num_tries += 1
        if num_tries > 2 * num_trials:
            raise ValueError(
                "Too many unsuccessful tries to compute metric.")

        # Handle case where only one class is included by indices.
        new_indices = np.random.choice(indices, size=len(indices))
        score = score_func(ground_truth[new_indices],
                             predictions[new_indices])
        scores.append(score)
        num_successes += 1

    mean = np.mean(scores)
    scores.sort()
    # Computed using basic bootstrap
    lower = 2 * mean - scores[
        int(((1 + confidence_level) / 2) * num_successes)]
    upper = 2 * mean - scores[
        int(((1 - confidence_level) / 2) * num_successes)]

    result = {'lb': lower, 'mean': mean, 'ub': upper}
    return result


In [ ]:
train_pred = model.predict(train_x)
valid_pred = model.predict(valid_x)

tr = {}
vr = {}

vr_ci = {}
for metric in metric_list:
    train_score = metric_list[metric][0](train_gt, train_pred)
    valid_score = metric_list[metric][0](valid_gt, valid_pred)
    # print(f'{metric}: {valid_score}')

    tr[metric] = train_score
    vr[metric] = valid_score

    ci = generate_confidence_interval(valid_gt, valid_pred,
                                 metric_list[metric][0],
                                 num_trials=500,
                                 confidence_level=0.95)
    
    vr_ci[metric] = ci
    
    
joint_df = pd.DataFrame({'Train': tr, 'Valid': vr})
joint_df

In [ ]:
valid_ci_df = pd.DataFrame.from_dict(vr_ci, orient='index')
valid_ci_df

# Examine the Actual Results

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(valid_gt, valid_pred, s=2)
plt.hlines(1, 0.9975, 1.0025, linewidth=1)
plt.vlines(1, 0.9975, 1.0025, linewidth=1)
plt.xlabel('Ground Truth')
plt.ylabel('Predictions')